In [2]:
import re
import nltk
import pandas as pd
from sklearn.datasets import load_files
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import urllib.request as ur
nltk.download('wordnet')
nltk.download('omw-1.4')
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
def removeDigits(List):
    
    answer = []
    for s in List:
      res=""
      for char in s :
        if not char.isdigit() and char!=' ':
              res+=char
      if len(res)>=2:
        answer.append(res)
    return ' '.join(answer)

In [4]:
#Fetch the contents from website and rmove all the unessary character,punctuations
def fetch_content(url):
      
      if "http://" and "https://" not in url:
        url="http://"+url
      X=[]

      try:
        page = requests.get(url)
        
        soup = BeautifulSoup(page.text, 'html.parser')

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        # get text
        text = soup.get_text()
        text=re.sub('[^A-Za-z0-9]+', ' ', text).strip()
        text=str(text).replace('\n',' ').split(' ')   
      
        for i in text:
          if i:
            X.append(i)
      except:
        X=[]
      
      documents = []
      stemmer = WordNetLemmatizer()
     
      if X:
        #Remove the digits from string:
        document = removeDigits(X)
      
        # Remove all the special characters
        document = re.sub(r'\W', ' ', document)
          
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
          
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
          
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
          
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
          
        # Converting to Lowercase
        document = document.lower()
          
        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)

        return document

In [ ]:
contents=[]
Label=[]
counter=0
ValidLink=[]
ValidLabel=[]

import pickle

# define dictionary



counter=0
print(len(labels))
for label,Urls in zip(labels,documents):
    content=fetch_content(Urls)
  
    counter+=1
   
    print(counter)
    if content:
      ValidLink.append(Urls)
      ValidLabel.append(label)
      contents.append(content)
      
      # create a binary pickle file 
      with open("FinalLabel", "wb") as fp:
        pickle.dump(ValidLabel,fp)
      with open("FinalLink", "wb") as fp:
        pickle.dump(ValidLink,fp)
      with open("Fianlcontents", "wb") as fp:
        pickle.dump(contents,fp)
      
      



In [6]:
import pickle


#Label = pickle.loads(f)


with open("Fianlcontents", "rb") as fp:   # Unpickling

  contents = pickle.load(fp)

with open("FinalLabel", "rb") as fp:   # Unpickling

  Label = pickle.load(fp)





In [9]:


  
#contents=[]
#Label=[]
#counter=0

 



# dictionary of lists  
dict = {'text': contents,'labels':Label}  
df = pd.DataFrame(dict)

df.to_csv('textdata.csv') 

nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=1, max_df=1.0, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(contents).toarray()


print(X[0])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def calculate_metrics(predicted, actual):
    TP, FP, TN, FN = 0, 0, 0, 0
    for i in range(len(predicted)):
        if   (predicted[i] == 0) & (actual[i] == 0):
            TP += 1
        elif (predicted[i] == 0) & (actual[i] == 1):
            FP += 1
        elif (predicted[i] == 1) & (actual[i] == 1):
            TN += 1
        else:
            FN += 1

    accuracy  = (TP + TN) / (TP + FP + TN + FN) 
    precision = (TP) / (TP + FP) 
    recall    = (TP) / (TP + FN) 
    f1_score  = (2 * precision * recall) / (precision + recall)
    
    return  precision, recall

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Label,test_size=0.35,shuffle=True)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(x_train, y_train) 
y_pred = classifier.predict(x_test)

from sklearn import metrics
from sklearn.metrics import precision_recall_curve,recall_score,precision_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[792  60]
 [ 61 353]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       852
           1       0.85      0.85      0.85       414

    accuracy                           0.90      1266
   macro avg       0.89      0.89      0.89      1266
weighted avg       0.90      0.90      0.90      1266

0.9044233807266983


In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train, y_train)

y_pred = naive_bayes.predict(x_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[801  51]
 [109 305]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       852
           1       0.86      0.74      0.79       414

    accuracy                           0.87      1266
   macro avg       0.87      0.84      0.85      1266
weighted avg       0.87      0.87      0.87      1266

0.8736176935229067
